In [1]:
import turicreate as tc

In [2]:
people = tc.SFrame('people_wiki.sframe')

In [3]:
people['word_count'] = tc.text_analytics.count_words(people['text'])

In [4]:
tfidf = tc.text_analytics.tf_idf(people['word_count'])
people['tfidf'] = tfidf

In [5]:
people.head(3)

Columns:
	URI	str
	name	str
	text	str
	word_count	dict
	tfidf	dict

Rows: 3

Data:
+-------------------------------+----------------+
|              URI              |      name      |
+-------------------------------+----------------+
| <http://dbpedia.org/resour... | Digby Morrell  |
| <http://dbpedia.org/resour... | Alfred J. Lewy |
| <http://dbpedia.org/resour... | Harpdog Brown  |
+-------------------------------+----------------+
+-------------------------------+-------------------------------+
|              text             |           word_count          |
+-------------------------------+-------------------------------+
| digby morrell born 10 octo... | {'melbourne': 1.0, 'colleg... |
| alfred j lewy aka sandy le... | {'time': 1.0, 'each': 1.0,... |
| harpdog brown is a singer ... | {'time': 1.0, 'honored': 1... |
+-------------------------------+-------------------------------+
+-------------------------------+
|             tfidf             |
+-------------------------------+
| {'melbourne': 3.8914310119... |
| {'time': 1.325334207420049... |
| {'time': 1.325334207420049... |
+-------------------------------+
[3 rows x 5 columns]

# quiz 1 & 2
quiz_1 : Top word count words for Elton John

quiz_2 : Top TF-IDF words for Elton John

In [14]:
Elton = people[people['name'] == 'Elton John']
Elton['word_count'].stack().sort("X2", ascending=False).head(5)

X1,X2
the,27.0
in,18.0
and,15.0
of,13.0
a,10.0


In [16]:
top_count = Elton[['tfidf']].stack('tfidf', new_column_name=['word','count'])

In [18]:
top_count.sort('count',ascending=False).head(5)

word,count
furnish,18.38947183999428
elton,17.482320270031995
billboard,17.30368095754203
john,13.93931279239831
songwriters,11.250406447031539


# distance
Question 3
The cosine distance between 'Elton John's and 'Victoria Beckham's articles (represented with TF-IDF) falls within which range?

Question 4
The cosine distance between 'Elton John's and 'Paul McCartney's articles (represented with TF-IDF) falls within which range?

Question 5
Who is closer to 'Elton John', 'Victoria Beckham' or 'Paul McCartney'?

In [19]:
Victoria_Beckham = people[people['name'] == 'Victoria Beckham']
Paul_McCartney   = people[people['name'] == 'Paul McCartney']

In [22]:
print('quiz_3')
print(tc.distances.cosine(Elton['tfidf'][0],Victoria_Beckham['tfidf'][0]))

print(' ')
print('quiz_4')
print(tc.distances.cosine(Elton['tfidf'][0],Paul_McCartney['tfidf'][0]))

print(' ')
print('quiz_5')
print('Paul McCartney is closer to Elton')

quiz_3
0.9567006376655429
 
quiz_4
0.8250310029221779
 
quiz_5
Paul McCartney is closer to Elton


# knn_model
Question 6
Who is the nearest cosine-distance neighbor to 'Elton John' using raw word counts?

Question 7
Who is the nearest cosine-distance neighbor to 'Elton John' using TF-IDF?

Question 8
Who is the nearest cosine-distance neighbor to 'Victoria Beckham' using raw word counts?

Question 9
Who is the nearest cosine-distance neighbor to 'Victoria Beckham' using TF-IDF?

In [30]:
knn_model_raw = tc.nearest_neighbors.create(people,features=['word_count'], label='name', distance='cosine')

Starting brute force nearest neighbors model training.

Validating distance components.

Initializing model data.

Initializing distances.

Done.

In [32]:
knn_model_raw.query(Elton)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 15.633ms     |

| Done         |         | 100         | 158.947ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Elton John,2.220446049250313e-16,1
0,Cliff Richard,0.16142415258967036,2
0,Sandro Petrone,0.16822542751041114,3
0,Rod Stewart,0.16832716558706107,4
0,Malachi O'Doherty,0.177315545978884,5


In [31]:
knn_model_tfidf = tc.nearest_neighbors.create(people, features=['tfidf'], label='name', distance='cosine')

Starting brute force nearest neighbors model training.

Validating distance components.

Initializing model data.

Initializing distances.

Done.

In [33]:
knn_model_tfidf.query(Elton)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 22.938ms     |

| Done         |         | 100         | 173.894ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Elton John,-2.220446049250313e-16,1
0,Rod Stewart,0.7172196678927374,2
0,George Michael,0.7476009989692847,3
0,Sting (musician),0.7476719544306141,4
0,Phil Collins,0.7511932487904706,5


In [34]:
print(knn_model_raw.query(Victoria_Beckham))
print(knn_model_tfidf.query(Victoria_Beckham))

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 9.029ms      |

...

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 9.509ms      |

| Done         |         | 100         | 142.073ms    |

+--------------+---------+-------------+--------------+

+-------------+---------------------+------------------------+------+
| query_label |   reference_label   |        distance        | rank |
+-------------+---------------------+------------------------+------+
|      0      |   Victoria Beckham  | 1.1102230246251565e-16 |  1   |
|      0      |    David Beckham    |   0.5481696102632145   |  2   |
|      0      | Stephen Dow Beckham |   0.7849867068283364   |  3   |
|      0      |        Mel B        |   0.8095855234085036   |  4   |
|      0      |    Caroline Rush    |    0.81982642291868    |  5   |
+-------------+---------------------+------------------------+------+
[5 rows x 4 columns]

